<a href="https://colab.research.google.com/github/HowlHowlHowl/NLP/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# GITHUB REPO: https://github.com/iiscleap/Coswara-Data
# PAPER: https://arxiv.org/pdf/2005.10548.pdf
!git clone https://github.com/iiscleap/Coswara-Data.git

In [ ]:
import os
import subprocess
import glob
from google.colab import drive
drive.mount('/content/drive')

coswara_data_dir = os.path.abspath('.')+ '/Coswara-Data'  # Percorso locale della directory dei dati di Coswara
extracted_data_dir = os.path.abspath('.')+ '/Extracted-Data'
drive_data_dir = os.path.abspath('.')+ '/drive/MyDrive/Extracted-Data'

print("Extracted data dir", extracted_data_dir)
print("Coswara data dir", coswara_data_dir)

if not os.path.exists(coswara_data_dir):
    raise FileNotFoundError("Check the Coswara dataset directory!")

if not os.path.exists(extracted_data_dir):
    os.makedirs(extracted_data_dir)

dirs_extracted = set(map(os.path.basename, glob.glob('{}/202*'.format(extracted_data_dir))))
dirs_all = set(map(os.path.basename, glob.glob('{}/202*'.format(coswara_data_dir))))

dirs_to_extract = list(set(dirs_all) - dirs_extracted)

for d in dirs_to_extract:
    tar_cmd = 'cat {}/{}/*.tar.gz.* | tar -xvz -C {}'.format(coswara_data_dir, d, extracted_data_dir)
    print("Executing command:", tar_cmd)
    p = subprocess.Popen(tar_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = p.communicate()
    print("Standard output:", stdout.decode())
    print("Standard error:", stderr.decode())

    test_file_path = os.path.join(extracted_data_dir, 'test.txt')

print("Extraction process complete!")

In [ ]:
!cp -r /content/Extracted-Data /content/drive/MyDrive/Extracted-Data

In [ ]:
import shutil
# Delete the repository after data extraction
# shutil.rmtree(coswara_data_dir)

Una volta eseguiti i passaggi precedenti NON devono essere compilati mai più

In [ ]:
from scipy.stats import kurtosis, skew

def statistic_obj(array):
    q3, q1 = np.percentile(array, [75, 25])
    return dict({
        'mean': float(np.mean(array)),
        'std_dev': float(np.std(array)),
        'min': float(np.min(array)),
        'max': float(np.max(array)),
        'median': float(np.median(array)),
        'skew': float(skew(array, axis=None)),
        'kurt': float(kurtosis(array, axis=None)),
        'perc25': float(q1),
        'perc75': float(q3),
        'root_mean_sqr': float(np.sqrt(np.mean(array ** 2))),
        'iqr': float(q3 - q1)
    })


In [ ]:
def extract_statistical_features(vec, name='None'):
    # prendiamo i primi 13 array e per ogni variante di mfcc
    if name == 'mfcc' or name == 'mfcc_d' or name == 'mfcc_d2':
        vec_features = []
        # ne calcoliamo gli indicatori statistici
        for i, array in enumerate(vec[0:13]):
            # scarto interquartile
            vec_features.append(statistic_obj(array))
        return vec_features
    else:
        return statistic_obj(vec)

In [ ]:
import librosa
import numpy as np
import re

def extract_features(audio, sr, category_name):
    onset_env = librosa.onset.onset_strength(y=audio, sr=sr)

    trim, index = librosa.effects.trim(y=audio)
    duration = librosa.get_duration(y=trim)
    # print('duration', duration)

    # times = librosa.times_like(onset_env, sr=sr)
    onset_frames = librosa.onset.onset_detect(onset_envelope=onset_env, sr=sr)

    # print('onset_frames:', onset_frames)

    # Scommentare 1 comentare 2, run di quello grosso e di nuovo scommentare 2 e commentre 1
    #tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
    tempo = librosa.feature.rhythm.tempo(onset_envelope=onset_env, sr=sr)
    # print('tempo:', tempo[0])

    period = np.argmax(librosa.stft(audio))
    # print('period:', period)

    rmse = librosa.feature.rms(y=audio)[0]
    rmse_stat = extract_statistical_features(rmse, 'rmse')
    # print('rmse:', np.shape(rmse))

    sc = librosa.feature.spectral_centroid(y=audio)
    sc_stat = extract_statistical_features(sc, 'sc')
    # print('spectral centroid:' + str(np.shape(sc[0])) + ' ' + str(np.shape(sc)))

    roll_off = librosa.feature.spectral_rolloff(y=audio, sr=sr, roll_percent=0.85)
    roll_off_stat = extract_statistical_features(roll_off, 'roll_off')
    # print('roll_off:' + str(np.shape(roll_off[0])) + ' ' + str(np.shape(roll_off)))

    zc = librosa.feature.zero_crossing_rate(y=audio)
    zc_stat = extract_statistical_features(zc, 'zc')
    # print('zero crossing:' + str(np.shape(zc[0])) + ' ' + str(np.shape(zc)))

    # la funzione mfcc restituisce matrici che stenderemo e ne calcoleremo gli indicatori statistici
    mfcc = librosa.feature.mfcc(y=audio, sr=sr)
    mfcc_feat = extract_statistical_features(mfcc, 'mfcc')

    mfcc_delta = librosa.feature.delta(mfcc)
    mfcc_feat_d = extract_statistical_features(mfcc_delta, 'mfcc_d')

    mfcc_delta2 = librosa.feature.delta(mfcc, order=2)
    mfcc_feat_d2 = extract_statistical_features(mfcc_delta2, 'mfcc_d2')

    features = {
        category_name + 'duration': float(duration),
        category_name + 'tempo': float(tempo[0]),
        category_name + 'period': float(period),
        category_name + 'onset': len(onset_frames),
        category_name + 'rmse': rmse_stat,
        category_name + 'sc': sc_stat,
        category_name + 'roll_off': roll_off_stat,
        category_name + 'zc': zc_stat,
        category_name + 'mfcc': mfcc_feat,
        category_name + 'mfcc_d': mfcc_feat_d,
        category_name + 'mfcc_d2': mfcc_feat_d2,
    }

    # This piece of code expand these arrays from
    # 'name': { 'stat_1': stat_1, ..., 'stat_n': stat_n }
    # to
    # 'name_stat_1': stat_1, ...,'name_stat_n': stat_n
    for name in [category_name + 'rmse', category_name + 'sc', category_name + 'roll_off', category_name + 'zc']:
        for key, value in features[name].items():
            # Split the original key name into separate parts
            parts = re.findall(r'[a-zA-Z]+|\d+', key)
            # Build the new key name by joining the parts with underscores
            new_key = "_".join([name] + parts)
            # Add the new key and value to the output dictionary
            features[new_key] = value
        del features[name]

    # This code perform the flattening of the mfcc-like matrices
    for name in [category_name + 'mfcc', category_name + 'mfcc_d', category_name + 'mfcc_d2']:
        for i, element in enumerate(features[name]):
            for key in element:
                new_key = name+'_'+str(i)+'_'+key
                features[new_key] = element[key]
        del features[name]

    return features

In [ ]:
def extraction(prefix, file_path):
    audio, sr = librosa.core.load(file_path)
    audio, _ = librosa.effects.trim(audio)
    return extract_features(audio, sr, prefix)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Percorso completo del file di output nel tuo Google Drive
output_file = os.path.join(os.path.abspath('.') + '/drive/MyDrive/Extracted-Data/', 'dataset_nuovo.json')
# Percorso completo del file di errori nel tuo Google Drive
errors_file = os.path.join(os.path.abspath('.') + '/drive/MyDrive/Extracted-Data/', 'errors.txt')

def save_dataset(dataset):
    with open(output_file, 'w') as f:
        json.dump(dataset, f, indent=4)

def save_errors(errors):
    with open(errors_file, 'w') as f:
        json.dump(dataset, f, indent=4)

In [ ]:
# Percorso della directory principale
main_dir = os.path.abspath('.') + '/drive/MyDrive/Extracted-Data/Extracted-Data'

# Conteggio del numero totale di sottodirectory 202___
total_subdirs_202 = sum(1 for _ in os.scandir(main_dir) if _.is_dir() and _.name.startswith("202"))


# NON ESEGUIRE

In [ ]:
import os
import json

# Metadata information: age, gender, location (country, state/province),
# current health status (healthy / exposed / cured / infected) and the presence
# of co-morbidity (pre-existing medical conditions).

dataset = []
errors = []

# Variabili contatore per il progresso
processed_subdirs_202 = 0
processed_samples = 0

# Iterazione su tutte le sottodirectory 202___
for root, dirs, files in os.walk(main_dir):
    for subdir in dirs:
        if subdir.startswith("202"):  # Solo se il nome della sottodirectory inizia con "202"
            processed_subdirs_202 += 1
            print("> Parsing subdir", subdir, f"({processed_subdirs_202}/{total_subdirs_202})")
            subdir_path = os.path.join(root, subdir)

            # Conteggio del numero totale di sottocartelle all'interno di questa sottodirectory 202___
            total_sample_dirs = sum(1 for _ in os.scandir(subdir_path) if _.is_dir())
            processed_sample_dirs = 0
            # Iterazione su tutte le sottocartelle
            for subroot, subdirs, subfiles in os.walk(subdir_path):
                for subsubdir in subdirs:
                    processed_sample_dirs += 1
                    sample_dir_path = os.path.join(subroot, subsubdir)

                    # Inizializzazione delle variabili
                    covid = None
                    audio_features = {}

                    print(f"> > Parsing {os.path.basename(sample_dir_path)} - sample: {processed_sample_dirs}/{total_sample_dirs}")

                    # Estrazione dei dati da ogni file nella sottocartella Sample_id
                    for file_name in os.listdir(sample_dir_path):
                        file_path = os.path.join(sample_dir_path, file_name)

                        if file_name.endswith('.json'):
                          try:
                            with open(file_path, 'r') as f:
                                data = json.load(f)
                                covid = data['covid_status']
                                audio_features.update({
                                    'age': data['a'],
                                    'gender': data['g']
                                })

                          except Exception as e:
                                err = (f"< < Error processing metadata of {os.path.basename(sample_dir_path)} -> { file_path.split('/')[7]}:\n\t{str(e)}" )
                                print(err)
                                errors.append(err)
                        else:
                            # print("calling extraction on", file_name)
                            # Esegui l'estrazione delle features per ogni file
                            try:
                                # Esegui l'estrazione delle features per ogni file
                                feature_name = os.path.splitext(file_name)[0]
                                audio_features.update(extraction(feature_name+'_', file_path))
                            except Exception as e:
                                err = (f"< < Error processing sample {subdir} - { file_path.split('/')[7]}: file {file_path.split('/')[8]}\n\t{str(e)}" )
                                print(err)
                                errors.append(err)
                    processed_samples += 1
                    dataset.append({
                        'sample': os.path.basename(sample_dir_path),
                        'covid': covid,
                        'features': audio_features
                    })

            # Print per avvisare che hai finito con la subdir
            print(f"{processed_samples} processed samples")
            print(f"> Finished parsing subdir: {subdir}... Saving dataset and errors.")

            save_dataset(dataset)
            save_errors(errors)

# Stampa il numero totale di subdir 202___
print("Total 202___ subdirectories:", total_subdirs_202)


In [ ]:
with open('/dataset.json', 'r') as file:
    raw_data = json.load(file)

In [ ]:
from google.colab import drive
import json
import os
drive.mount('/content/drive')
data_file = os.path.join(os.path.abspath('.') + '/drive/MyDrive/Extracted-Data/', 'dataset.json')

with open(data_file, 'r') as file:
    raw_data = json.load(file)

Mounted at /content/drive


In [ ]:
# Inizializza un dizionario per contare le occorrenze di ciascun valore di 'covid'
conteggio_covid = {}

for elemento in raw_data:
    valore_covid = elemento['covid']
    if valore_covid in conteggio_covid:
        conteggio_covid[valore_covid] += 1
    else:
        conteggio_covid[valore_covid] = 1

# Stampa i conteggi di ciascun valore di 'covid'
print("Conteggi di 'covid':", json.dumps(conteggio_covid, indent=4))

Conteggi di 'covid': {
    "healthy": 1481,
    "positive_mild": 426,
    "positive_moderate": 165,
    "no_resp_illness_exposed": 282,
    "positive_asymp": 90,
    "recovered_full": 146,
    "resp_illness_not_identified": 159
}


Qui riduciamo da 7 classi a 2 (perdendone 1 ovvero resp_illness_not_identified = #159) ovvero positivi e negativi

In [ ]:
positives = []
negatives = []

for el in raw_data:
    if isinstance(el, dict):  # Verifica se l'elemento è un dizionario
        if "covid" in el and isinstance(el["covid"], str):  # Verifica se la chiave "covid" è presente e se il suo valore è una stringa
            if el["covid"] in ['healthy', 'recovered_full', 'no_resp_illness_exposed']:
                el["covid"] = "negatives"
                negatives.append(el)
            elif el["covid"] != 'resp_illness_not_identified':
                el["covid"] = "positives"
                positives.append(el)
        else:
            print("Warning: Element does not contain 'covid' key or 'covid' value is not a string:", el)
    else:
        print("Warning: Element is not a dictionary:", el)

print(f"Positives = {len(positives)}")
print(f"Negatives = {len(negatives)}")
positives.extend(negatives)
data = positives
print('Original data was ',len(raw_data))
print('New data is' , len(data))

Positives = 681
Negatives = 1909
Original data was  2749
New data is 2590


Qui eliminamo i sample che contengono almeno un NaN fra le features (tutti i dati devono essere uguali in formato)

In [ ]:
import pandas as pd

# Estrai le features e il target da ogni elemento del dataset
features_list = []
target_list = []

# Filtra i sample che non contengono valori NaN nelle featuresimport json
filtered_data = []
for entry in data:
    has_missing_values = any(pd.isna(value) for key, value in entry['features'].items() if key != 'gender' and isinstance(value, (int, float)))
    if not has_missing_values:
        filtered_data.append(entry)
print("New data without NaN", len(filtered_data))

New data without NaN 2492


Qui eliminamo i dati che non hanno lo stesso numero di feature degli altri (file audio corrotti/ impossibile elaborarli, ad esdempio alcuni audio hanno lunghezza 0)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 1. Estrai le features e il target
features_list = []
target_list = []

for entry in filtered_data:
  # Converti il valore del campo 'gender' da stringa a 0 per 'female' e 1 per 'male'
    gender_value = 1 if entry['features']['gender'] == 'male' else 0

    # Aggiorna il campo 'gender' nel dizionario delle features
    entry['features']['gender'] = gender_value

    features_list.append(entry['features'])
    target_list.append(entry['covid'])

# Lista degli indici da rimuovere
to_remove = []
for i, c in enumerate(features_list):
    if len(c) != 4295:
        to_remove.append(i)

print("Original sample list length:", len(features_list))
print("Original target list length:", len(target_list))

# Rimuovi gli elementi dalla lista in ordine inverso
for idx in sorted(to_remove, reverse=True):
    del features_list[idx]
    del target_list[idx]

print("New sample list length:", len(features_list))
print("New target list length:", len(target_list))


features = pd.DataFrame(features_list)

Original sample list length: 2492
Original target list length: 2492
New sample list length: 2432
New target list length: 2432


In [ ]:
from sklearn.preprocessing import LabelEncoder
# Codifica le etichette del target
label_encoder = LabelEncoder()
target_encoded = label_encoder.fit_transform(target_list)
# Ottieni i nomi delle classi originali
classes_names = label_encoder.classes_

In [ ]:
import numpy as np
# Calcola la matrice di correlazione tra le features
correlation_matrix = features.corr()

Raggruppiamo in insiemi le features che sono correlate fra loro almeno al 90% in quanto ridontanti

In [ ]:
from collections import defaultdict

# Identifica i set di features strettamente correlate
corr_threshold = 0.90
corr_sets = defaultdict(set)  # Dizionario per memorizzare i set di features correlate

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > corr_threshold:
            corr_sets[i].add(correlation_matrix.columns[i])
            corr_sets[i].add(correlation_matrix.columns[j])

In [ ]:
n_features_to_keep = 1000  # Specifica il numero di feature da mantenere
n_top_to_keep = 500 # numero di feature con cui terminare
proba_threshold = 0.2
xgboost_threshold = 0.6

Eliminamo suddette features lasciandone 1 a set

In [ ]:
# Crea un insieme per tenere traccia delle feature selezionate
selected_features = set()

# Ciclo su ogni insieme di features correlate
for corr_set in corr_sets.values():
    # Calcola la correlazione di ogni feature nel set con il target
    corr_with_target = {feature: abs(features[feature].corr(pd.Series(target_encoded))) for feature in corr_set}
    # Ordina le feature in base alla correlazione con il target
    sorted_features = sorted(corr_with_target.items(), key=lambda x: x[1], reverse=True)
    # Seleziona solo le prime n_features_to_keep
    top_features = [feature for feature, _ in sorted_features[:n_features_to_keep]]
    # Aggiungi le top feature selezionate all'insieme selected_features
    selected_features.update(top_features)


In [ ]:

# Calcola la correlazione delle feature selezionate con il target
correlation_with_target = features[selected_features].apply(lambda x: abs(x.corr(pd.Series(target_encoded))))
# Calcola la correlazione delle feature selezionate con il target
correlation_with_target = correlation_with_target.nlargest(n_features_to_keep)

# Ottieni il nome delle feature selezionate
selected_features = correlation_with_target.index

# Aggiorna il DataFrame features_selected per includere solo le feature selezionate
features_selected = features[selected_features]

# Stampa la correlazione con il target per le feature selezionate
print(correlation_with_target)

In [ ]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import SVC
import numpy as np

# Dividi il dataset in set di training, validation e test
X_train_val, X_test, y_train_val, y_test = train_test_split(features_selected, target_encoded, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

SVM SETUP

In [ ]:
# Addestramento con validazione incrociata per trovare i migliori iperparametri
parameters = {'kernel':('linear', 'rbf'), 'C':[0.1, 1, 10]}
svm = SVC(probability=True)
clf = GridSearchCV(svm, parameters, cv=5)
clf.fit(X_train_val, y_train_val)

# Valutazione sul set di validazione
best_model = clf.best_estimator_
y_pred_val_prob = best_model.predict_proba(X_val)[:, 1]  # Probabilità della classe positiva
y_pred_val = (y_pred_val_prob >= proba_threshold).astype(int)  # Applicazione della soglia
val_report = classification_report(y_val, y_pred_val, target_names=classes_names)
print("Classification Report on validation set:")
print(val_report)

# Addestramento sul set di addestramento e di validazione combinato
best_model.fit(X_train_val, y_train_val)

# Valutazione finale sul set di test
y_pred_test_prob = best_model.predict_proba(X_test)[:, 1]  # Probabilità della classe positiva
y_pred_test = (y_pred_test_prob >= proba_threshold).astype(int)  # Applicazione della soglia
test_report = classification_report(y_test, y_pred_test, target_names=classes_names)
print("Classification Report on test set:")
print(test_report)

LR SETUP

In [ ]:
# Addestramento con validazione incrociata per trovare i migliori iperparametri
parameters = {'C':[0.1, 1, 10], 'penalty': ['l1', 'l2']}
lr = LogisticRegression(solver='liblinear', max_iter=1000)
clf = GridSearchCV(lr, parameters, cv=5)
clf.fit(X_train_val, y_train_val)

# Valutazione sul set di validazione
best_model = clf.best_estimator_
y_pred_val_prob = best_model.predict_proba(X_val)[:, 1]  # Probabilità della classe positiva
y_pred_val = (y_pred_val_prob >= proba_threshold).astype(int)  # Applicazione della soglia
val_report = classification_report(y_val, y_pred_val, target_names=classes_names)
print("Classification Report on validation set:")
print(val_report)

# Addestramento sul set di addestramento e di validazione combinato
best_model.fit(X_train_val, y_train_val)

# Valutazione finale sul set di test
y_pred_test_prob = best_model.predict_proba(X_test)[:, 1]  # Probabilità della classe positiva
y_pred_test = (y_pred_test_prob >= proba_threshold).astype(int)  # Applicazione della soglia
test_report = classification_report(y_test, y_pred_test, target_names=classes_names)
print("Classification Report on test set:")
print(test_report)

XGBoost SETUP

In [ ]:

# E' solo per ricordaci quali parametri possiamo settare in xgboost, non viene utilizzato
possible_parameters = {
    'objective': 'multi:softmax',  # Funzione obiettivo per problemi di classificazione multiclasse
    'num_class': 2,  # Numero di classi nel tuo problema
    'n_estimators': 200,  # Numero di alberi da addestrare
    'learning_rate': 0.2,  # Tasso di apprendimento
    'max_depth': 8,  # Massima profondità dell'albero
    'min_child_weight': 1,  # Peso minimo richiesto per suddividere un nodo
    'gamma': 0,  # Parametro di riduzione della perdita
    'subsample': 0.8,  # Frazione di campioni da utilizzare per addestrare ciascun albero
    'colsample_bytree': 0.8,  # Frazione di colonne da utilizzare per addestrare ciascun albero
    'reg_alpha': 0,  # Parametro di regolarizzazione L1
    'reg_lambda': 1,  # Parametro di regolarizzazione L2
    'random_state': 42  # Seed per la riproducibilità dei risultati
}

# Definizione del modello XGBoost
xgboost_model = xgb.XGBClassifier()

# Definizione dei parametri da esplorare
param_grid = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300],
}

# Ricerca dei migliori iperparametri con GridSearchCV
grid_search = GridSearchCV(estimator=xgboost_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_val, y_train_val)

# Visualizzazione dei migliori iperparametri trovati
print("Best parameters found:", grid_search.best_params_)

# Valutazione sul set di validazione
best_xgboost_model = grid_search.best_estimator_
y_pred_val_prob = best_xgboost_model.predict_proba(X_val)[:, 1]  # Probabilità della classe positiva
y_pred_val = (y_pred_val_prob >= xgboost_threshold).astype(int)  # Applicazione della soglia
val_report = classification_report(y_val, y_pred_val, target_names=classes_names)
print("Classification Report on validation set:")
print(val_report)

# Valutazione sul set di test
y_pred_test_prob = best_xgboost_model.predict_proba(X_test)[:, 1]  # Probabilità della classe positiva
y_pred_test = (y_pred_test_prob >= xgboost_threshold).astype(int)  # Applicazione della soglia
test_report = classification_report(y_test, y_pred_test, target_names=classes_names)
print("Classification Report on test set:")
print(test_report)

creazione della **curva ROC** per tutti i tre modelli che abbiamo visto:

- La curva ROC è utile per valutare le prestazioni di un classificatore binario. Mostra il tasso di veri positivi (sensibilità) contro il tasso di falsi positivi (1-specificità). Un modello ideale avrà un'area sotto la curva (AUC) di ROC pari a 1, mentre un modello casuale avrà un'AUC di 0.5.

- Non ha senso per il nostro scopo calcolarci la curva CMC

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

def plot_roc_curve(model, X_val, y_val):
    if hasattr(model, "predict_proba"):
        y_pred_prob = model.predict_proba(X_val)[:, 1]
    else:
        raise ValueError("Model doesn't have predict_proba method.")

    fpr, tpr, thresholds = roc_curve(y_val, y_pred_prob)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.show()

# Plot ROC curve for Logistic Regression
plot_roc_curve(best_model_lr, X_val, y_val)

# Plot ROC curve for SVM
plot_roc_curve(best_model_svm, X_val, y_val)

# Plot ROC curve for XGBoost
plot_roc_curve(best_model_xgboost, X_val, y_val)